In [ ]:
import numpy as np

In [208]:
data = pd.read_excel("MTND2.xlsx")
data.head()


,STT,GF,TBS,Natri Benzoate,Abs 260,Abs 265,Abs 270,Abs 275,Abs 280,Abs 285,Abs 290
0,1,3.0,3.0,3.0,0.333,0.452,0.584,0.625,0.534,0.240,0.052
1,2,3.0,3.0,2.0,0.301,0.423,0.556,0.608,0.526,0.241,0.052
2,4,7.0,7.0,2.0,0.575,0.848,1.143,1.283,1.128,0.534,0.113
3,5,3.0,3.0,1.5,0.283,0.405,0.537,0.592,0.513,0.237,0.052
4,6,3.0,3.0,1.0,0.263,0.386,0.519,0.577,0.503,0.234,0.050


In [209]:
data = data.drop(["STT"], axis = 1)
data = data.sample(frac=1)
data.head()

,GF,TBS,Natri Benzoate,Abs 260,Abs 265,Abs 270,Abs 275,Abs 280,Abs 285,Abs 290
20,0.5,2.0,0.5,0.117,0.156,0.201,0.229,0.214,0.138,0.052
19,0.5,1.5,0.5,0.103,0.135,0.172,0.192,0.176,0.108,0.044
0,3.0,3.0,3.0,0.333,0.452,0.584,0.625,0.534,0.240,0.052
38,1.0,2.5,1.5,0.198,0.256,0.322,0.351,0.313,0.177,0.055
3,3.0,3.0,1.5,0.283,0.405,0.537,0.592,0.513,0.237,0.052


In [210]:
# Split data into train and test data
data_train = data[:int(0.7 * len(data))].values
data_test = data[int(0.7 * len(data)):].values
# Convert data table to X and Y matricses
X_train = data_train[:,0:3].astype(float) 
Y_train = data_train[:,3:].reshape(-1, 7).astype(float)
X_test = data_test[:,0:3].astype(float) 
Y_test = data_test[:,3:].reshape(-1, 7).astype(float)
# Using principal component regression
D = np.dot(Y_train.T, Y_train)
U, S, vh = np.linalg.svd(D, full_matrices=True)
d = (np.diag(S)) / np.sum(np.diag(S)) * 100
f = vh[:, :5]
Y_j = np.dot(Y_train, f)
F = np.dot(np.dot(np.linalg.inv(np.dot(Y_j.T, Y_j)), Y_j.T), X_train)
F_j = np.dot(f, F)
# Find X_pred from Y_test
X_pred = np.dot(Y_test, F_j)

(30, 7)


In [211]:
D = np.dot(Y_train.T, Y_train)
U, S, vh = np.linalg.svd(D, full_matrices=True)
d = (np.diag(S)) / np.sum(np.diag(S)) * 100
f = vh[:, :5]
Y_j = np.dot(Y_train, f)
F = np.dot(np.dot(np.linalg.inv(np.dot(Y_j.T, Y_j)), Y_j.T), X_train)
F_j = np.dot(f, F)
X_pred = np.dot(Y_test, F_j)

In [212]:
X_pred = np.dot(Y_test, F_j)
print(X_train)
print(X_pred)

[[0.5 2.  0.5]
 [0.5 1.5 0.5]
 [3.  3.  3. ]
 [1.  2.5 1.5]
 [3.  3.  1.5]
 [2.5 2.5 1.5]
 [2.5 2.5 2.5]
 [1.5 2.5 2. ]
 [2.5 1.  1.5]
 [2.5 3.  2. ]
 [0.5 3.  0.5]
 [2.5 1.  2.5]
 [1.  1.  1. ]
 [7.  7.  2. ]
 [2.5 2.5 2. ]
 [2.5 1.  1. ]
 [2.  2.  1.5]
 [2.5 1.  2. ]
 [0.5 1.  0.5]
 [3.  3.  1. ]
 [0.5 0.5 0.5]
 [0.5 0.5 2.5]
 [3.  3.  2. ]
 [0.5 0.5 1. ]
 [1.  1.  0.5]
 [2.  2.  2. ]
 [2.5 2.5 1. ]
 [2.5 2.5 0.5]
 [2.5 3.  1.5]
 [1.  2.5 0.5]]
[[0.50300949 2.01552162 0.48140239]
 [0.51207015 1.39289501 0.36255387]
 [3.1100948  2.85014156 2.55163234]
 [1.04696502 2.56300602 1.76158545]
 [3.10679683 2.88961922 1.3276096 ]
 [2.4128124  2.71593223 1.58887055]
 [2.55567881 2.51742208 2.54225305]
 [1.15446623 2.59244369 2.14783228]
 [2.41032104 1.00040114 1.61879216]
 [2.63755331 2.99550279 2.12949538]
 [0.54799754 2.8217412  0.44141495]
 [2.47946904 0.96065624 2.63649728]
 [0.99051805 0.94710849 0.82740283]
 [6.80021981 6.9792252  1.97858975]
 [2.5166877  2.53187343 2.15026861]
 [2.34000

In [213]:
cost = 1 / (2*X_train.shape[0]) * np.sum(np.abs(X_train * 10 - X_pred * 10))
cost_ls = []

In [214]:
for i in range (10000):
    X_pred = np.dot(Y_train, F_j)
    cost = 1 / (2*X_train.shape[0]) * np.sum(np.abs(X_train - X_pred))
    d_theta = (1/X_train.shape[0])*np.dot(Y_train.T, X_train - X_pred)
    F_j = F_j - 0.001*d_theta
    cost_ls.append(cost)

In [215]:
cost_ls

[0.1516502896967598,
 0.1516502899001072,
 0.1516502901036505,
 0.15165029030738994,
 0.15165029051132567,
 0.15165029071545763,
 0.15165029091978607,
 0.1516502911243113,
 0.15165029132903335,
 0.15165029153395235,
 0.1516502917390687,
 0.15165029194438215,
 0.15165029214989337,
 0.15165029235560212,
 0.1516502925615088,
 0.15165029276761324,
 0.151650292973916,
 0.15165029318041717,
 0.15165029338711675,
 0.15165029359401488,
 0.15165029380111164,
 0.15165029400840763,
 0.15165029421590298,
 0.15165029442359748,
 0.15165029463149118,
 0.1516502948395845,
 0.15165029504787805,
 0.1516502952563712,
 0.1516502954650644,
 0.15165029567395816,
 0.15165029588305232,
 0.1516502960923469,
 0.15165029630184237,
 0.15165029651153886,
 0.1516502967214362,
 0.1516502969315347,
 0.15165029714183537,
 0.15165029735233715,
 0.15165029756304063,
 0.15165029777394642,
 0.15165029798505406,
 0.15165029819636386,
 0.1516502984078763,
 0.15165029861959123,
 0.15165029883150916,
 0.1516502990436298,
 0.1

In [216]:
X_pred2 = np.dot(Y_test, F_j)
print(X_pred2)
print(X_test)

[[2.5392639  3.11300417 0.66340973]
 [0.45191087 0.63821825 1.17627459]
 [2.10050638 1.93705397 0.50236633]
 [1.09843991 2.5495702  1.22218155]
 [0.480075   0.45950507 2.75792167]
 [2.64091675 2.94084868 2.68284978]
 [2.06826046 1.99735068 0.93148569]
 [0.57755201 2.42006283 0.48622588]
 [0.3941446  0.80448048 0.88494561]
 [2.44162707 1.13033235 0.67001843]
 [3.17141286 2.86837439 0.4008965 ]
 [1.52314556 2.79662889 0.66595999]
 [1.1849812  2.57610238 3.13922488]
 [0.45048615 0.52604622 1.31651468]]
[[2.5 3.  0.5]
 [0.5 0.5 2. ]
 [2.  2.  0.5]
 [1.  2.5 1. ]
 [0.5 0.5 3. ]
 [2.5 3.  2.5]
 [2.  2.  1. ]
 [0.5 2.5 0.5]
 [0.5 1.  1. ]
 [2.5 1.  0.5]
 [3.  3.  0.5]
 [1.2 1.5 1. ]
 [1.5 2.5 2.5]
 [0.5 0.5 1.5]]


In [217]:
cost_test = abs(((X_pred2 * 10 - X_test * 10) / (X_test * 10)) * 100)
cost_test


array([[ 1.57055601,  3.76680557, 32.68194628],
       [ 9.61782603, 27.64364963, 41.18627058],
       [ 5.02531903,  3.14730128,  0.47326695],
       [ 9.84399121,  1.98280804, 22.21815493],
       [ 3.98500064,  8.09898655,  8.06927775],
       [ 5.63666996,  1.97171059,  7.31399123],
       [ 3.41302303,  0.13246588,  6.85143122],
       [15.51040191,  3.19748694,  2.75482496],
       [21.17107985, 19.55195234, 11.50543903],
       [ 2.3349173 , 13.03323468, 34.00368683],
       [ 5.71376213,  4.38752033, 19.82069984],
       [26.92879673, 86.44192606, 33.40400144],
       [21.00125357,  3.04409529, 25.56899513],
       [ 9.90276976,  5.20924393, 12.23235458]])

In [218]:
for i in range (cost_test.shape[0]):
    for j in cost_test[i, :]:
        if (j > 30):
            print(X_test[i])

[2.5 3.  0.5]
[0.5 0.5 2. ]
[2.5 1.  0.5]
[1.2 1.5 1. ]
[1.2 1.5 1. ]
